In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df=pd.read_csv("../data/spot_5min_features.csv",parse_dates=["datetime"])
df=pd.set_index("datetime").sort_index
df=df.copy()
print(df.head())
print(df.columns)

ValueError: Missing column provided to 'parse_dates': 'datetime'

In [4]:
df = pd.read_csv("../data/spot_5min_features.csv")
print(df.columns)


Index(['open', 'high', 'low', 'close', 'volume', 'ret_1', 'ret_5', 'ema_20',
       'ema_50', 'ema_diff', 'volatility_20', 'volatility_100', 'hl_range',
       'volume_z', 'sentiment_pressure', 'future_price', 'basis',
       'delta_proxy', 'gamma_proxy', 'vega_proxy', 'theta_proxy'],
      dtype='object')
